In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, shutil
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!ls
!cd /content
!rm -fr darknet
!git clone https://github.com/AlexeyAB/darknet/
% cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!apt update
!apt-get install libopencv-dev
!make &> compile.log

In [ ]:
!cp ../drive/MyDrive/Colab\ Notebooks/CV003/obj.data data
!cp ../drive/MyDrive/Colab\ Notebooks/CV003/obj.names data
!cp ../drive/MyDrive/Colab\ Notebooks/CV003/train.txt data
!cp ../drive/MyDrive/Colab\ Notebooks/CV003/valid.txt data
!mkdir data/obj
# !cp ../drive/MyDrive/Colab\ Notebooks/CV003/obj/* data/obj
!cp ../drive/MyDrive/Colab\ Notebooks/CV003/yolo-obj.cfg cfg
!cp ../drive/MyDrive/Colab\ Notebooks/CV003/*.weights backup
!mkdir test_images
!cp ../drive/MyDrive/test_images/* test_images

In [ ]:
!ls data/obj | wc -l

## Testing the Models

In [ ]:
### Predictions
!./darknet detector test data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_1000.weights test_images/test03.jpeg -thresh 0.2

## Get Metrics

In [ ]:
# Cross-Validation ONLY
valid_file = open('data/valid.txt', 'r')
valid_images = valid_file.readlines()
valid_file.close()

for img_path in valid_images:
  formats = ['jpg', 'txt']
  image_filename = img_path[9:-5]
  
  for fmt in formats:
    source = os.path.join('../drive/MyDrive/Colab Notebooks/CV003/obj', '{}.{}'.format(image_filename, fmt))
    dest = os.path.join('data/obj', '{}.{}'.format(image_filename, fmt))
    shutil.copyfile(source, dest)

In [ ]:
!mkdir output

In [ ]:
### Cross Validation map
for i in range(1000, 9000, 1000):
  command_1 = './darknet detector map data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_{}.weights -points 0 > output/output{}_iou50.txt'.format(i,i)
  command_2 = './darknet detector map data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_{}.weights -iou_thresh 0.75 -points 0 > output/output{}_iou75.txt'.format(i,i)

  os.system(command_1)
  os.system(command_2)



In [ ]:
!cp output/* ../drive/MyDrive/Colab\ Notebooks/Results_CV003/

In [ ]:
!./darknet detector map data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_7000.weights -iou_thresh 0.75 -points 0

In [ ]:
!./darknet detector map data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_8000.weights -iou_thresh 0.75 -points 0

In [ ]:
!rm -r output